In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from numba import cuda, vectorize
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from functools import wraps
from time import time

In [43]:
df=pd.read_csv('../input/train-wala/aug_train.csv')
df.head(5)

In [42]:
#Let's Make A Function
def train():
    df=pd.read_csv('../input/train-wala/aug_train.csv')
    df=df[['enrollee_id','city_development_index','training_hours','target']]
    df.dropna()
    df['city_development_index']=df['city_development_index'].map(lambda x: "{:.0f}".format(x))
    df['target']=df['target'].map(lambda x: "{:.0f}".format(x))
    return df

def train_array():
    df=train()
    return np.float32(df)

@vectorize(['float32(float32,float32)'],target='cuda')
def add_ufunc(x,y):
    return x+y

def cuda_operation():
    """Performs Vectorized Opearations on GPU"""
    
    x=train_array()
    y=train_array()
    
    print('Movement Calculations From CPU to GPU!!!')
    x_device=cuda.to_device(x)
    y_device=cuda.to_device(y)
    out_device=cuda.device_array(shape=(x_device.shape[0],x_device.shape[1]),dtype=np.float32)
    print(x_device)
    print(x_device.shape)
    print(x_device.dtype)
    
    print('Calculating On GPU')
    add_ufunc(x_device,y_device,out=out_device)
    
    out_host=out_device.copy_to_host()
    print(f"Calculations from GPU {out_host}")
cuda_operation()    